## Zadanie domowe: morfologiczna gra w życie – John Conway

### Wykorzystanie operacji LUT w przekształceniu trafi, nie trafi
  - Szybszą metodą wykonania transformacji trafi, nie trafi może być operacja LUT.
  - Technika polega na zakodowaniu wyników wszystkich interesujących  konfiguracji, a następnie podczas przetwarzania wykorzystania operacji LUT.
  - Dla otoczenia 3x3 możliwe jest 512 różnych konfiguracji.
  - Aby praktycznie zrealizować operacje, każdej konfiguracji należy nadać unikalny indeks. Jedną z możliwości jest pomnożenie elementu strukturalnego przez macierz (mnożenie odpowiednich elementów):
  ```
  [[1, 8,  64],
   [ 2, 16, 128],
   [ 4, 32, 256]]
  ```
  Przykładowo elementowi:
  ```
  [[1, 1, 0],
   [ 1, 0, 1],
   [ 1, 0, 1]]
  ```
  odpowiada liczba: 1(1) + 2(1) + 4(1) + 8(1) + 128(1) + 256(1) = 399.
  
### Przykład działania metody – detekcja punktów końcowych na obrazie.
  - założenie: punkt końcowy to punkt, który ma dokładnie jednego sąsiada,
  - zdefiniuj funkcję, która jako argument pobiera otoczenie, a zwraca 0 lub 1 w zależności od tego, czy rozpatrywany punkt jest końcowy np. dla sąsiedztwa 3×3 punkt będzie końcowy, jeżeli jest zapalony i ma tylko jednego sąsiada (czyli suma pikseli jest równa 2).
  - wygeneruj przekodowanie LUT.
  - wczytaj obraz szkielet.bmp (należy go przekształcić, aby uzyskać dwuwymiarową tablicę o wartościach 0-1). Wykorzystując wygenerowane przekodowanie LUT wykonaj detekcję zakończeń. Wyświetl obraz oryginalny i po przekodowaniu LUT.

### Gra w życie

Reguły gry w życie:
  - każdy piksel biały, który ma dwóch lub trzech sąsiadów (białych) przeżywa,
  - każdy piksel biały, który ma 0,1 lub więcej niż trzech sąsiadów (białych) nie przeżywa (głód lub przeludnienie),
  - jeżeli jakieś pole (czarne) sąsiaduje dokładnie z trzema pikselami białymi, to na tym polu ,,rodzi'' się nowy piksel biały.

Zadanie:
  - za pomocą mechanizmu LUT (opisanego wcześniej) należy zaimplementować morfologiczną gre w życie,
  - najważniejszym elementem jest funkcja opisująca reguły gry,
  - symulacje należny przeprowadzić dla plansz dostarczonych w pliku gra.py,
  - dobrze jest wykonać kilka iteracji – zobaczyć jak zmienia się kształt,
  - inne ciekawe kształty do znalezienia w internecie.


In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os


if not os.path.exists("szkielet.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/10_Morphology/szkielet.bmp --no-check-certificate

szkielet = cv2.imread("szkielet.bmp", cv2.IMREAD_GRAYSCALE)


plt.imshow(szkielet, 'gray')
plt.axis("off")
plt.show()

szkielet_bin = np.where(szkielet < 3, 0, 1)

plt.imshow(szkielet_bin, 'gray')
plt.axis("off")
plt.show()


In [ ]:
def count(surr):
  summ = 0
  for el1 in surr:
    for el2 in el1:
      summ += el2

  if summ == 2:
    return 1
  else:
    return 0

szkielet_bin2 = np.pad(szkielet_bin, 1, mode='constant')
X,Y = szkielet_bin2.shape
result = np.zeros((X,Y))

for i in range(1,X-1):
    for j in range(1,Y-1):
        surr = szkielet_bin2[i-1:i+2,j-1:j+2]
        if surr[1,1] != 0:
          result[i,j] = count(surr)
        else:
          result[i,j] = 0;


plt.imshow(result, 'gray', vmin=0, vmax=1)
plt.axis('off')
plt.show()
 

In [ ]:
def count_lut(surr):

  summ = 0
  lut_tab = np.array([[1,8,64],[2,16,128],[4,32,256]])
  surr2 = surr * lut_tab

  for el1 in surr2:
    for el2 in el1:
      summ += el2
  return summ

def count_lut_bits(summ):

  number = np.binary_repr(summ)
  bits = 0
  i = 0

  for el in number:
    if i!= 4:
      if el == '1':
        bits += 1      
    i += 1
  return bits



# def game_of_life(board):

#   change = True
#   board2 = np.pad(board, 1, mode='constant')
#   X,Y = board2.shape
#   result = np.zeros((X,Y))

#   while change:

#     change = False
#     for i in range(1,X-1):
#         for j in range(1,Y-1):
#             surr = board2[i-1:i+2,j-1:j+2]
#             count = count_lut(surr)
#             bits = count_lut_bits(count)

#             if surr[1,1] == 0:
#               if bits == 3:
#                 result[i,j] = 1
#                 change = True             

#             else:
#               if bits != 2 and bits != 3:
#                 result[i,j] = 0    
#                 change = True        
       
#     print(result)
#   return result

def game_of_life_one_iter(board):

  X,Y = board.shape
  result = board.copy()

  for i in range(1,X-1):
      for j in range(1,Y-1):

          surr = board[i-1:i+2,j-1:j+2]
          count = count_lut(surr)
          bits = count_lut_bits(count)

          if surr[1,1] == 0:
            if bits == 3:
              result[i,j] = int(1)
        
          else:
            if bits != 2 and bits != 3:
              result[i,j] = int(0)  

  return result.astype(bool)


In [ ]:
plansza1 = np.array([
    [0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0],
    [0,0,0,1,0,0,1,0,0,0],
    [0,0,0,1,1,1,1,0,0,0],
    [0,0,1,0,0,0,0,1,0,0],
    [0,0,1,0,1,1,0,1,0,0],
    [0,0,1,0,0,0,0,1,0,0],
    [0,0,0,1,1,1,1,0,0,0],
    [0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0]], np.uint8)

plt.imshow(plansza1, "gray")
plt.show()

final_plansza = game_of_life_one_iter(plansza1)
plt.imshow(final_plansza, "gray")
plt.show()

for i in range(10):
  final_plansza = game_of_life_one_iter(final_plansza)
  plt.imshow(final_plansza, "gray")
  plt.show()


In [ ]:
plansza2 = np.array([
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0],
    [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0],
    [0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]], np.uint8)

plt.imshow(plansza2, "gray")
plt.show()
final_plansza = game_of_life_one_iter(plansza2)
plt.imshow(final_plansza, "gray")
plt.show()

for i in range(4):
  final_plansza = game_of_life_one_iter(final_plansza)
  plt.imshow(final_plansza, "gray")
  plt.show()
